![](../images/rivacon_frontmark_combined_header.svg)

# Equity forward curves

In [ ]:
import pyvacon.analytics as analytics
import datetime as dt
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import math
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline

## Definition of an equity forward curve
An equity forward curve provides the functionality to compute forward values of an equity underlying at arbitrary future times. The forward value of an equity underlying at time $T$ is given by

$$F_0 = (S_0 - I)^{(r-q)T}$$

where $S_0$ is the current spot price, $I$ is the present value of cash dividends, $r$ is the (risk-free) interest rate and $q$ is the borrow rate.


Hence, in order to setup an equity forward curve we use the classical forward relationship using
- spot value,
- interest rate curve given as discount curve ,
- borrow curve given as discount curve (this curve describes the yield from borrow/lending the underlying or the yield spread for repos),
- dividend table.

To setup an equity forward curve one therefore needs to construct these components. How we can construct a discount curve is discussed in the [discount curves](discount_curves.ipynb) notebook. How to construct a dividend table is shown in the next paragraph.

## Creating an equity forward curve

### Setup the specification

#### Creating a dividend table

A dividend table stores dividends together with their payment and ex-dates and a tax factor. Here, a dividend is split into two parts: A cash dividend and a dividend yield. In the pricing models the dividend yields are interpreted as relative values to the spot at the dividend ex-date.

In order to construct a dividend table, we furthermore need an object id as well as a reference date. A reference date is needed for all objects representing market data. Most of the functions provided by market data objects do also get a calculation/valuation date and some logic is applied if this date is after the reference date. For a discount curve, if the valuation date is after the reference date, the forward discount factor is returned. Dates which enter analytics objects must be analytics ptimes.

Our dividend table is given the name *div_table* and is defined as an *analytics.DividendTable* object which we provide with the information described before. 

In [ ]:
object_id = "TEST_DIV"
refdate = analytics.ptime(2017,1,1,0,0,0)

ex_dates = converter.createPTimeList(refdate, [dt.datetime(2018,3,29), dt.datetime(2019,3,29), dt.datetime(2020,3,29), dt.datetime(2021,3,29)])
pay_dates = converter.createPTimeList(refdate, [dt.datetime(2018,4,1), dt.datetime(2019,4,1), dt.datetime(2020,4,1), dt.datetime(2021,4,1)])
tax_factors = analytics.vectorDouble([1.0, 1.0, 1.0, 1.0])
div_yield = analytics.vectorDouble([0, 0.005, 0.01, 0.01])
div_cash = analytics.vectorDouble([3.0, 2.0, 1.0, 0.0])

div_table=analytics.DividendTable(object_id, refdate, ex_dates, div_yield, div_cash, tax_factors, pay_dates)
#help(analytics.DividendTable)

#### Creating discount and borrowing curves

We need a list of dates and a list of discount factors. For the purpose of illustration we just use zero borrow and interest rates. The alternative way would be to create a list of dates and discount factors analogous to the methodology described in the [discount curves](discount_curves.ipynb) notebook. This can be practiced in the exercise provided.

The discount- and borrowing curves are then constructed using the created dates and discount factors.

In [ ]:
dates = converter.createPTimeList(refdate,[0, 2*365])
dc_df = analytics.vectorDouble([1.0,1.0])
bc_df = analytics.vectorDouble([1.0,1.0])

dc = analytics.DiscountCurve(object_id, refdate,dates, dc_df, enums.DayCounter.ACT365_FIXED, enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)
bc = analytics.DiscountCurve(object_id, refdate,dates, bc_df, enums.DayCounter.ACT365_FIXED, enums.InterpolationType.HAGAN_DF, enums.ExtrapolationType.NONE)
#help(analytics.DiscountCurve)
#for d in dates:
#    print(d.to_string())

#### Setting the spot price

The last input information we need to construct the equity forward curve is the current spot price $S_0$ which we set manually as the variable *spot*.

In [ ]:
spot = 100.0

### Setup the equity forward curve
Our equity forward curve is given the name *forward_curve* and is defined as an *analytics.EquityForwardCurve* object which we provide with the information described before. 

In [ ]:
forward_curve = analytics.EquityForwardCurve(refdate, spot, dc, bc, div_table)
#help(analytics.EquityForwardCurve)

## Computing forward values
As the forward curve inherits from the *BaseDatedCurve* class, the computation of a forward value is provided by the same interface as for all curves, the *value* function.

In [ ]:
fwd_value = forward_curve.value(refdate, converter.getLTime(600, refdate))
#print(converter.getLTime(600,refdate).to_string())
#help(forward_curve.value)
print(fwd_value)

## Plotting forward curve

In [ ]:
mkt_plot.curve(forward_curve, range(600), refdate)
#help(mkt_plot.curve)

## Exercise

- Create a forward curve based on the dividend table defined above and with a flat borrow rate (3%) and flat discount curve (1.5%).